# Final Project - Super Blocks

In [2]:
# Import
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import floor
import patsy
import statsmodels.api as sm
from scipy.stats import ttest_ind

## Introduction and Background

### Many cities and neighbourhoods in the United States, such as Los Angeles have been designed for extensive car use. While these models might have worked in the past, this dynamic is beginning to change as the result of overpopulation, rideshares and automation among other factors. Urban neighborhoods are no longer just places of living for large amounts of people, but rather an intertwined mesh of commercial activity, transport and residences (3). This does not fit the originally intended model - it is not efficient, causes pollution, and prevents children from being able to play in the streets.

### In Barcelona(2), city planners realized this problem and implemented a system where they blocked off certain urban areas to vehicle traffic, opening them up for commercial and community use. The results were astounding - with an increase shown in community engagement along with a decrease in pollution. Economic benefits have also been shown for the local businesses in these "superblocks" (4,5). As a result, we have decided to create a model that can figure out the best blocks to block out for such activity, geospatially and temporally.

### We will be using the number of restaurants, coffee shops, bars and transit stations to categorize the 'walkability' of different blocks, building a model analagous to walkscore (1). Furthermore, we will be using the Yelp API to temporally analyze foot traffic based on numbers and timing of reviews and checkins.

##### Referencess:
  ######  1)https://www.walkscore.com/professional/research.php
   ###### 2)https://www.theguardian.com/cities/2016/may/17/superblocks-rescue-barcelona-spain-plan-give-streets-back-residents
   ###### 3)https://www.wired.com/2017/04/brilliant-simplicity-new-yorks-new-times-square/ 
   ###### 4)http://krqe.com/2017/03/06/city-councilor-wants-wider-sidewalks-to-help-businesses-impacted-by-art/
   ###### 5)http://www.nyc.gov/html/dot/downloads/pdf/dot-economic-benefits-of-sustainable-streets.pdf

## Data Description

'FY2014 Ridership_Trolley_Sept2013Booking.csv' has the information about the number of people getting on and off the trolley for each ride for Sept2013.

'FY2014 Ridership_Trolley_Sept2013_Stops.csv' has the information about the location and other information about the trolley locations in San Diego.

## Data Cleaning / Pre-processing

### Active Businesses (Restaurants, cafe's etc) 

In [33]:
#Importing the original file for active business data in SD

fname = "Business.csv"
business_df = pd.read_csv(fname)

In [34]:
#Importing the original file for active business data in SD

fname1 = "Stop_Counts.csv"
stops_df = pd.read_csv(fname1)

In [35]:
#Importing the original file for active business data in SD

fname2 = "Parking_Counts.csv"
parking_df = pd.read_csv(fname2)

In [36]:
#Removing unneccesary columns and renaming to layman terms, filtering all business into places that serve food/drinks

business_df = business_df[['doing_bus_as_name','zip','naics_description','lat','lon']]
business_df = business_df.loc[(business_df['naics_description'] == 'full-service restaurants') |
               (business_df['naics_description'] == 'cafeterias') | 
               (business_df['naics_description'] == 'food services & drinking places') |
               (business_df['naics_description'] == 'limited-service eating places') |
               (business_df['naics_description'] == 'limited-service restaurants')  |
               (business_df['naics_description'] == 'mobile food services') |
               (business_df['naics_description'] == 'drinking places (alcoholic beverages)') |
               (business_df['naics_description'] == 'snack & nonalcoholic beverage bars')]
business_df.rename(columns = {'doing_bus_as_name':'Business title','naics_description':'Type of Place'}, inplace=True)
business_df = business_df.reset_index(drop=True)

In [37]:
#Adding the 5 columns for the different time brackets

business_df.insert(2,'AM early','Null')
business_df.insert(3,'AM peak','Null')
business_df.insert(4,'Mid-day','Null')
business_df.insert(5,'PM peak','Null')
business_df.insert(6,'PM late','Null')

In [38]:
#generalising the different categories into 3 categories: Only Food, Food & Drinks, Only Drinks

business_df.loc[(business_df['Type of Place'] == 'mobile food services') |
                (business_df['Type of Place'] == 'cafeterias') |
                (business_df['Type of Place'] == 'snack & nonalcoholic beverage bars') |
                (business_df['Type of Place'] == 'limited-service eating places') |
                (business_df['Type of Place'] == 'limited-service restaurants') 
                , 'Type of Place'] = 'Only Food'

business_df.loc[(business_df['Type of Place'] == 'full-service restaurants') 
                , 'Type of Place'] = 'Food & Drinks'

business_df.loc[(business_df['Type of Place'] == 'drinking places (alcoholic beverages)') |
                (business_df['Type of Place'] == 'food services & drinking places') 
                , 'Type of Place'] = 'Only Drinks'

In [39]:
#Assigning values to different time brackets by assuming foot traffic according to the type of place 

business_df.loc[(business_df['Type of Place'] == 'Only Food'), 
                ('AM early','AM peak','Mid-day','PM peak','PM late')] = ('3','25','20','18','5')
business_df.loc[(business_df['Type of Place'] == 'Food & Drinks'),
                ('AM early','AM peak','Mid-day','PM peak','PM late')] = ('7','28','25','35','25')
business_df.loc[(business_df['Type of Place'] == 'Only Drinks'),
                ('AM early','AM peak','Mid-day','PM peak','PM late')] = ('27','6','3','27','30') 

In [40]:
#Assigning a total score to all three categories (summ of scores of all time brackets)

business_df.loc[(business_df['Type of Place'] == 'Only Food'), 
                'Total Score'] = '71'
business_df.loc[(business_df['Type of Place'] == 'Food & Drinks'),
                'Total Score'] = '120'
business_df.loc[(business_df['Type of Place'] == 'Only Drinks'),
                'Total Score'] = '93'

In [41]:
business_df

,Business title,zip,AM early,AM peak,Mid-day,PM peak,PM late,Type of Place,lat,lon,Total Score
0,c r e a m,92115-1939,3,25,20,18,5,Only Food,32.767243,-117.096294,71
1,snow cones y raspados,92113-2911,3,25,20,18,5,Only Food,32.697974,-117.096250,71
2,jalapeno taco shop,92104-2047,7,28,25,35,25,Food & Drinks,32.748683,-117.126968,120
3,tacos el campechano inc,91950-1121,3,25,20,18,5,Only Food,32.697974,-117.096250,71
4,up2you cafe llc,92111-5000,3,25,20,18,5,Only Food,32.718370,-117.157817,71
5,awash ethiopian restaurant,92104-1102,7,28,25,35,25,Food & Drinks,32.677079,-117.107167,120
6,gueros taco shop,92102-4019,3,25,20,18,5,Only Food,32.890137,-117.150877,71
7,the fire spot,92111-2315,7,28,25,35,25,Food & Drinks,32.773069,-117.156144,120
8,up2you cafe llc,92111-1545,27,6,3,27,30,Only Drinks,33.016118,-117.075608,93
9,cold beers & cheeseburgers,92101-6910,7,28,25,35,25,Food & Drinks,32.708791,-117.160357,120


In [ ]:
business_df.to_csv('restaurants_data.csv')

### Transit Stops Dataset
The following cells take the original dataset and clean it so that we get the cleaned data, NOT MEANT TO BE RUN since we already have the cleaned dataset

#Code which we used to clean the original dataset which is way too big for Github 

trips = pd.read_csv('FY2014 Ridership_Trolley_Sept2013Booking.csv')

# Removing unnecessary columns: all we need is stop_id, passengers getting off on the station 
# and time_arrival to get the time split for the number of people getting off at a station at a time_period

trips = trips[['STOP_ID', 'PASSENGERS_OFF', 'TIME_ACTUAL_ARRIVE']]
trips.columns = ['stop_id', 'count', 'time']
trips

##### clean the data, removing any na rows we will also remove all rows that have value 0

trips.dropna(how='any')
trips = trips[trips['count'] != 0]
trips

# Using stop_id we can connect with lat/long. We will be grouping by pole_id to create a table with 
# the following columns: (stop_id, latitude, longitude, count_am_early, count_am_peak, count_midday, 
# count_pm_early, count_pm_late, count_daily)

#### Getting location data for all the stops using another dataset
stop_locs = pd.read_csv('FY2014 Ridership_Trolley_Sept2013_Stops.csv')
stop_locs = stop_locs[['STOP_ID', 'LAT', 'LON']]
stop_locs

stop_locs.columns = ['stop_id', 'latitude', 'longitude']

stops_df = trips.merge(stop_locs, how='left')
stops_df = stops_df.dropna(how='any')
stops_df

##### grouping by stop_id and aggregating over the count values

stop_counts = stops_df.groupby(['stop_id']).agg('count')

##### remove unnecessary columns

stop_counts = stop_counts['time'] 
stop_counts = stop_counts.to_frame()
stop_counts['stop_id'] = stop_counts.index
stop_counts = stop_counts.merge(stop_locs, how='left')
stop_counts.columns = ['total_count', 'stop_id', 'latitude', 'longitude']
stop_counts

##### We need to now find the number of days in the transactions dataset
##### We will be using this in order to get the count of transactions PER DAY

stops_df['time'] = pd.to_datetime(stops_df['time'])
dates = stops_df['time']
am_early_d = {}
am_peak_d = {}
midday_d = {}
pm_peak_d = {}
pm_late_d = {}

##### The time ranges for which we have split the transit ridership data are:
AM_Early = 12AM-6AM
AM_Late = 6AM-9AM
..
..


#### Classify the time slot based on the times during the day
def classify(x): 
    hour = x.time().hour
    if hour <=6:
        return 'am_early'
    elif hour <=9:
        return 'am_peak'
    elif hour <=14:
        return 'midday'
    elif hour <=19:
        return 'pm_peak'
    else:
        return 'pm_late'
stops_df['time_slot'] = stops_df['time'].apply(classify)
stops_df

##### Merging the two datasets by key value and column name

def checkSeriesColumn(s, col):
    val = False
    for row in s.keys().to_series().str.contains(col): 
        if(row == True):
            val = True
    return val

### Setting the values of the transactions for each time period during the day
def set_temporal_counts(p_id):
    v_counts = stops_df.loc[stops_df['stop_id'] == p_id]['time_slot'].value_counts(dropna=False)
    stop_counts.loc[stop_counts['stop_id'] == p_id,'am_early'] = 0 if not checkSeriesColumn(v_counts, 'am_early') else v_counts['am_early']
    stop_counts.loc[stop_counts['stop_id'] == p_id,'am_peak'] =  0 if not checkSeriesColumn(v_counts, 'am_peak') else v_counts['am_peak']
    stop_counts.loc[stop_counts['stop_id'] == p_id,'midday'] =  0 if not checkSeriesColumn(v_counts, 'midday') else v_counts['midday']
    stop_counts.loc[stop_counts['stop_id'] == p_id,'pm_peak'] =  0 if not checkSeriesColumn(v_counts, 'pm_peak') else v_counts['pm_peak']
    stop_counts.loc[stop_counts['stop_id'] == p_id,'pm_late'] = 0 if not checkSeriesColumn(v_counts, 'pm_late') else v_counts['pm_late']

stop_counts['stop_id'].apply(set_temporal_counts)

stop_counts.to_csv('Stop_Counts.csv')

In [ ]:
#cleaned dataset for the transit spots
stops_df

### Parking meters 

###### Importing data and initial cleaning

In [19]:
#importing dataset
parking = pd.read_csv('treas_parking_payments_2017_datasd.csv')

In [20]:
#removing unnecessary columns: all we need is pole_id, time_start and meter_expire
parking = parking.drop(['uuid'], axis=1)
parking = parking.drop(['trans_amt'], axis=1)
parking = parking.drop(['pay_method'], axis=1)
parking = parking.drop(['meter_type'], axis=1)
parking = parking.drop(['meter_expire'], axis=1)

In [21]:
#clean the data, removing any na rows 
parking.dropna(how='any')

,pole_id,trans_start
0,SL-216,01/01/17 0:15
1,5-402,01/01/17 1:03
2,G-503,01/01/17 1:23
3,G-503,01/01/17 1:27
4,G-503,01/01/17 1:27
5,G-503,01/01/17 1:27
6,G-503,01/01/17 1:27
7,Mar-41,01/01/17 1:29
8,Mar-41,01/01/17 1:30
9,Apr-65,01/01/17 3:01


###### Connecting with other dataset (with lat/long pairs)

In [22]:
# 2.) using pole_id we can connect with lat/long. We will be grouping by pole_id to create
#     a table with the following columns:
#     (pole_id, latitude, longitude, count_am_early, count_am_peak, 
#        count_midday, count_pm_early, count_pm_late, count_daily)
# NOTE: we will need to decide whether to use raw numbers or averages of counts per section per day

In [23]:
park_loc = pd.read_csv('treas_parking_meters_loc_datasd.csv')

In [24]:
park_loc = park_loc[['pole', 'longitude', 'latitude']]
park_loc.columns = ['pole_id', 'longitude', 'latitude']
park_loc = park_loc.dropna(how='any')

In [25]:
park_df = parking.merge(park_loc, how='left')
park_df = park_df.dropna(how='any')
park_df

,pole_id,trans_start,longitude,latitude
0,SL-216,01/01/17 0:15,-117.162112,32.710495
1,5-402,01/01/17 1:03,-117.160212,32.709683
2,G-503,01/01/17 1:23,-117.159897,32.712561
3,G-503,01/01/17 1:27,-117.159897,32.712561
4,G-503,01/01/17 1:27,-117.159897,32.712561
5,G-503,01/01/17 1:27,-117.159897,32.712561
6,G-503,01/01/17 1:27,-117.159897,32.712561
12,AH-715,01/01/17 5:18,-117.157665,32.719824
13,7-1000,01/01/17 5:51,-117.158444,32.715845
14,7-1000,01/01/17 5:51,-117.158444,32.715845


In [26]:
park_counts = park_df.groupby(['pole_id']).agg('count')
park_counts = park_counts['trans_start'] #remove unnecessary columns
park_counts = park_counts.to_frame()
park_counts['pole_id'] = park_counts.index
park_counts = park_counts.merge(park_loc, how='left')
park_counts.columns = ['total_count', 'pole_id', 'longitude', 'latitude']
park_counts

,total_count,pole_id,longitude,latitude
0,237,1-1004,-117.163929,32.715904
1,237,1-1006,-117.163930,32.716037
2,228,1-1008,-117.163931,32.716169
3,208,1-1020,-117.161278,32.717890
4,239,1-1310,-117.163951,32.719024
5,254,1-1312,-117.163952,32.719161
6,217,1-1313,-117.163770,32.719298
7,286,1-1314,-117.163953,32.719453
8,224,1-1315,-117.163772,32.719571
9,238,1-1317,-117.163772,32.719707


In [ ]:
# we need to now find the number of days in the transactions dataset
# we will be using this in order to get the count of transactions PER DAY
park_df['trans_start'] = pd.to_datetime(park_df['trans_start'])
dates = park_df['trans_start']
am_early_d = {}
am_peak_d = {}
midday_d = {}
pm_peak_d = {}
pm_late_d = {}

def classify(x): 
    hour = x.time().hour
    if hour <=6:
        return 'am_early'
    elif hour <=9:
        return 'am_peak'
    elif hour <=14:
        return 'midday'
    elif hour <=19:
        return 'pm_peak'
    else:
        return 'pm_late'
park_df['time_slot'] = park_df['trans_start'].apply(classify)
park_df

In [ ]:
def checkSeriesColumn(s, col):
    val = False
    for row in s.keys().to_series().str.contains(col): 
        if(row == True):
            val = True
    return val
def set_temporal_counts(p_id):
    v_counts = park_df.loc[park_df['pole_id'] == p_id]['time_slot'].value_counts(dropna=False)
    park_counts.loc[park_counts['pole_id'] == p_id,'am_early'] = 0 if not checkSeriesColumn(v_counts, 'am_early') else v_counts['am_early']
    park_counts.loc[park_counts['pole_id'] == p_id,'am_peak'] =  0 if not checkSeriesColumn(v_counts, 'am_peak') else v_counts['am_peak']
    park_counts.loc[park_counts['pole_id'] == p_id,'midday'] =  0 if not checkSeriesColumn(v_counts, 'midday') else v_counts['midday']
    park_counts.loc[park_counts['pole_id'] == p_id,'pm_peak'] =  0 if not checkSeriesColumn(v_counts, 'pm_peak') else v_counts['pm_peak']
    park_counts.loc[park_counts['pole_id'] == p_id,'pm_late'] = 0 if not checkSeriesColumn(v_counts, 'pm_late') else v_counts['pm_late']
#     park_counts.loc[park_counts['pole_id'] == ]
#      park_counts.loc[park_counts['pole_id'] == p_id,'am_early'] = 0 if not checkSeriesColumn(v_counts, 'am_early') else v_counts['am_early']
#      park_counts.loc[park_counts['pole_id'] == p_id,'am_peak'] =  0 if not checkSeriesColumn(v_counts, 'am_peak') else v_counts['am_peak']
#     park_counts.loc[park_counts['pole_id'] == p_id,'midday'] =  0 if not checkSeriesColumn(v_counts, 'midday') else v_counts['midday']
#     park_counts.loc[park_counts['pole_id'] == p_id,'pm_peak'] =  0 if not checkSeriesColumn(v_counts, 'pm_peak') else v_counts['pm_peak']
#     park_counts.loc[park_counts['pole_id'] == p_id,'pm_late'] = 0 if not checkSeriesColumn(v_counts, 'pm_late') else v_counts['pm_late']


park_counts['pole_id'].apply(set_temporal_counts)




# v_counts = park_df.loc[park_df['pole_id'] == 'N-1003']['time_slot'].value_counts(dropna=False)
# park_counts.loc[park_counts['pole_id'] == 'N-1003','am_early'] = 0 if not checkSeriesColumn(v_counts, 'am_early') else v_counts['am_early']
# park_counts.loc[park_counts['pole_id'] == 'N-1003','am_peak'] =  0 if not checkSeriesColumn(v_counts, 'am_peak') else v_counts['am_peak']
# park_counts.loc[park_counts['pole_id'] == 'N-1003','midday'] =  0 if not checkSeriesColumn(v_counts, 'midday') else v_counts['midday']
# park_counts.loc[park_counts['pole_id'] == 'N-1003','pm_peak'] =  0 if not checkSeriesColumn(v_counts, 'pm_peak') else v_counts['pm_peak']
# park_counts.loc[park_counts['pole_id'] == 'N-1003','pm_late'] = 0 if not checkSeriesColumn(v_counts, 'pm_late') else v_counts['pm_late']
# park_counts.loc[park_counts['pole_id'] == 'N-1003']



In [ ]:
max_date = dates.max()
min_date = dates.min()
days_elapsed = (max_date - min_date).days + 1 #to round off this number
days_elapsed

In [ ]:
park_counts.to_csv("Parking_Counts.csv");

In [ ]:
# let's now get average counts of parked vehicles
# dividing by total days to give per daily expected counts 
park_counts['total_count'] = park_counts['total_count'].apply(lambda x: x / days_elapsed)
park_counts['am_early'] = park_counts['am_early'].apply(lambda x: x / days_elapsed)
park_counts['am_peak'] = park_counts['am_peak'].apply(lambda x: x / days_elapsed)
park_counts['midday'] = park_counts['midday'].apply(lambda x: x / days_elapsed)
park_counts['pm_peak'] = park_counts['pm_peak'].apply(lambda x: x / days_elapsed)
park_counts['pm_late'] = park_counts['pm_late'].apply(lambda x: x / days_elapsed)

In [ ]:
# we then multiply this amount by the mean number of people per vehicle
# as per https://www.rita.dot.gov/bts/sites/rita.dot.gov.bts/files/publications/highlights_of_the_2001_national_household_travel_survey/html/table_a15.html
ppl_per_vehicle = 1.63
park_counts['total_count'] = park_counts['total_count'].apply(lambda x: x * ppl_per_vehicle)
park_counts['am_early'] = park_counts['am_early'].apply(lambda x: x * ppl_per_vehicle)
park_counts['am_peak'] = park_counts['am_peak'].apply(lambda x: x * ppl_per_vehicle)
park_counts['midday'] = park_counts['midday'].apply(lambda x:  x * ppl_per_vehicle)
park_counts['pm_peak'] = park_counts['pm_peak'].apply(lambda x:  x * ppl_per_vehicle)
park_counts['pm_late'] = park_counts['pm_late'].apply(lambda x:  x * ppl_per_vehicle)
park_counts

In [ ]:
park_counts = pd.read_csv("Parking_Counts.csv");
####Rough Calculations for algorithm
long_max = park_counts['longitude'].max()
long_min = park_counts['longitude'].min()
lat_max = park_counts['latitude'].max()
lat_min = park_counts['latitude'].min()
lat_dif = lat_max - lat_min
long_dif = long_max - long_min
NUMBER_BLOCKS_ROOT = 10 #this means 100 blocks 10x10
lat_gap = lat_dif / NUMBER_BLOCKS_ROOT
long_gap = long_dif / NUMBER_BLOCKS_ROOT

def classify_blocks(s):
    park_counts.loc[park_counts['pole_id'] == s,'row'] =  (park_counts.loc[park_counts['pole_id'] == s,'latitude'] - lat_min) // lat_gap
    park_counts.loc[park_counts['pole_id'] == s,'col'] = (park_counts.loc[park_counts['pole_id'] == s,'longitude'] - long_min) // long_gap

park_counts['pole_id'].apply(classify_blocks)

park_counts

In [ ]:
print(long_max)
print(long_min)
print(long_dif)
print(lat_max)
print(lat_min)
print(lat_dif)
park_counts = park_counts.loc[park_counts['longitude'] != -180.0 ] #remove the outlier
long_min = park_counts['longitude'].min()
long_min

In [ ]:
park_counts.to_csv("Parking_Counts_grid.csv");

## Data Visualization

## Data Analysis and Results 

In [6]:
park_counts = pd.read_csv('Parking_Counts.csv')
park_counts = park_counts.loc[park_counts['longitude'] != -180.0 ] #remove the outlier
park_counts.rename(columns={'pole_id':'id'}, inplace= True)
park_counts['type'] = 'parking'
del park_counts['Unnamed: 0']
park_counts 

,total_count,id,longitude,latitude,midday,am_early,am_peak,pm_peak,pm_late,type
0,10.767183,1-1004,-117.163929,32.715904,5.096620,0.298451,2.066197,3.191127,0.114789,parking
1,11.283732,1-1006,-117.163930,32.716037,5.578732,0.286972,1.905493,3.455141,0.057394,parking
2,10.927887,1-1008,-117.163931,32.716169,5.050704,0.137746,2.261338,3.432183,0.045915,parking
3,10.044014,1-1020,-117.161278,32.717890,3.776549,0.160704,1.882535,4.166831,0.057394,parking
4,12.362746,1-1310,-117.163951,32.719024,7.105423,0.573944,3.420704,1.251197,0.011479,parking
5,12.718592,1-1312,-117.163952,32.719161,7.713803,0.321408,3.524014,1.124930,0.034437,parking
6,12.282394,1-1313,-117.163770,32.719298,7.426831,0.309930,3.546972,0.975704,0.022958,parking
7,13.326972,1-1314,-117.163953,32.719453,7.644930,0.516549,3.799507,1.354507,0.011479,parking
8,11.111549,1-1315,-117.163772,32.719571,6.784014,0.149225,3.122254,1.056056,0.000000,parking
9,11.203380,1-1317,-117.163772,32.719707,6.956197,0.195141,3.351831,0.700211,0.000000,parking


In [7]:
restaurants = pd.read_csv('Restaurant_Counts.csv')
restaurants.columns = ['id', 'title', 'zip', 'am_early', 'am_peak', 'midday', 'pm_peak', 'pm_late', 'type', 'latitude', 'longitude', 'total_count']
restaurants['total_count'] = restaurants['total_count'].apply( lambda x : x * 2/3)
restaurants['am_peak'] = restaurants['am_peak'].apply( lambda x : x * 2/3)
restaurants['am_early'] = restaurants['am_early'].apply( lambda x : x * 2/3)
restaurants['midday'] = restaurants['midday'].apply( lambda x : x * 2/3)
restaurants['pm_peak'] = restaurants['pm_peak'].apply( lambda x : x * 2/3)
restaurants['pm_late'] = restaurants['pm_late'].apply( lambda x : x * 2/3)
restaurants['id'] = restaurants['id'].apply(lambda x : ('R-' + str(x) ))
restaurants = restaurants[['id','type','title', 'am_early', 'am_peak', 'midday', 'pm_peak', 'pm_late', 'total_count','latitude', 'longitude']]

restaurants

,id,type,title,am_early,am_peak,midday,pm_peak,pm_late,total_count,latitude,longitude
0,R-0,Only Food,c r e a m,2.000000,16.666667,13.333333,12.000000,3.333333,46.666667,32.767243,-117.096294
1,R-1,Only Food,snow cones y raspados,2.000000,16.666667,13.333333,12.000000,3.333333,46.666667,32.697974,-117.096250
2,R-2,Food & Drinks,jalapeno taco shop,4.666667,18.666667,16.666667,23.333333,16.666667,79.333333,32.748683,-117.126968
3,R-3,Only Food,tacos el campechano inc,2.000000,16.666667,13.333333,12.000000,3.333333,46.666667,32.697974,-117.096250
4,R-4,Only Food,up2you cafe llc,2.000000,16.666667,13.333333,12.000000,3.333333,46.666667,32.718370,-117.157817
5,R-5,Food & Drinks,awash ethiopian restaurant,4.666667,18.666667,16.666667,23.333333,16.666667,79.333333,32.677079,-117.107167
6,R-6,Only Food,gueros taco shop,2.000000,16.666667,13.333333,12.000000,3.333333,46.666667,32.890137,-117.150878
7,R-7,Food & Drinks,the fire spot,4.666667,18.666667,16.666667,23.333333,16.666667,79.333333,32.773069,-117.156144
8,R-8,Only Drinks,up2you cafe llc,18.000000,4.000000,2.000000,18.000000,20.000000,62.000000,33.016118,-117.075608
9,R-9,Food & Drinks,cold beers & cheeseburgers,4.666667,18.666667,16.666667,23.333333,16.666667,79.333333,32.708791,-117.160357


In [8]:
transit = pd.read_csv('Stop_Counts.csv')
transit.rename(columns={'stop_id':'id'}, inplace= True)
transit['type'] = 'transit_stop'
del transit['Unnamed: 0'] 
transit

,total_count,id,latitude,longitude,am_early,am_peak,midday,pm_peak,pm_late,type
0,214,75000,32.54,-117.03,48.0,36.0,40.0,70.0,20.0,transit_stop
1,214,75000,32.54,-117.03,48.0,36.0,40.0,70.0,20.0,transit_stop
2,101,75002,32.56,-117.05,21.0,18.0,20.0,32.0,10.0,transit_stop
3,98,75003,32.56,-117.05,20.0,16.0,20.0,32.0,10.0,transit_stop
4,101,75004,32.57,-117.07,21.0,18.0,20.0,32.0,10.0,transit_stop
5,101,75005,32.57,-117.07,23.0,17.0,19.0,32.0,10.0,transit_stop
6,101,75006,32.59,-117.08,20.0,19.0,20.0,32.0,10.0,transit_stop
7,95,75007,32.59,-117.08,16.0,17.0,20.0,32.0,10.0,transit_stop
8,101,75008,32.60,-117.08,20.0,19.0,20.0,31.0,11.0,transit_stop
9,99,75009,32.60,-117.08,21.0,16.0,20.0,32.0,10.0,transit_stop


In [9]:
location_counts = restaurants.append(park_counts).append(transit)
# Setting a new index
#location_counts.index = idx # new ad hoc index
# location_counts.index = range(len(location_counts)) # set with list
# location_counts = location_counts.reset_index() # replace old w new
# location_counts.rename(columns={'index':'idx'}, inplace= True)
# location_counts.rename(columns={'id':'type_id'}, inplace= True)
# del location_counts['Unnamed: 0']
location_counts

,am_early,am_peak,id,latitude,longitude,midday,pm_late,pm_peak,title,total_count,type
0,2.000000,16.666667,R-0,32.767243,-117.096294,13.333333,3.333333,12.000000,c r e a m,46.666667,Only Food
1,2.000000,16.666667,R-1,32.697974,-117.096250,13.333333,3.333333,12.000000,snow cones y raspados,46.666667,Only Food
2,4.666667,18.666667,R-2,32.748683,-117.126968,16.666667,16.666667,23.333333,jalapeno taco shop,79.333333,Food & Drinks
3,2.000000,16.666667,R-3,32.697974,-117.096250,13.333333,3.333333,12.000000,tacos el campechano inc,46.666667,Only Food
4,2.000000,16.666667,R-4,32.718370,-117.157817,13.333333,3.333333,12.000000,up2you cafe llc,46.666667,Only Food
5,4.666667,18.666667,R-5,32.677079,-117.107167,16.666667,16.666667,23.333333,awash ethiopian restaurant,79.333333,Food & Drinks
6,2.000000,16.666667,R-6,32.890137,-117.150878,13.333333,3.333333,12.000000,gueros taco shop,46.666667,Only Food
7,4.666667,18.666667,R-7,32.773069,-117.156144,16.666667,16.666667,23.333333,the fire spot,79.333333,Food & Drinks
8,18.000000,4.000000,R-8,33.016118,-117.075608,2.000000,20.000000,18.000000,up2you cafe llc,62.000000,Only Drinks
9,4.666667,18.666667,R-9,32.708791,-117.160357,16.666667,16.666667,23.333333,cold beers & cheeseburgers,79.333333,Food & Drinks


In [10]:
long_max = location_counts['longitude'].max()
long_min = location_counts['longitude'].min()
lat_max = location_counts['latitude'].max()
lat_min = location_counts['latitude'].min()
lat_dif = lat_max - lat_min
long_dif = long_max - long_min
NUMBER_BLOCKS_ROOT = 10 #this means 100 blocks 10x10
lat_gap = lat_dif / NUMBER_BLOCKS_ROOT
long_gap = long_dif / NUMBER_BLOCKS_ROOT

def classify_blocks(s):
    location_counts.loc[location_counts['id'] == s,'row'] =  (location_counts.loc[location_counts['id'] == s,'latitude'] - lat_min) // lat_gap
    location_counts.loc[location_counts['id'] == s,'col'] = (location_counts.loc[location_counts['id'] == s,'longitude'] - long_min) // long_gap

location_counts['id'].apply(classify_blocks)

location_counts

,am_early,am_peak,id,latitude,longitude,midday,pm_late,pm_peak,title,total_count,type,row,col
0,2.000000,16.666667,R-0,32.767243,-117.096294,13.333333,3.333333,12.000000,c r e a m,46.666667,Only Food,3.0,7.0
1,2.000000,16.666667,R-1,32.697974,-117.096250,13.333333,3.333333,12.000000,snow cones y raspados,46.666667,Only Food,2.0,7.0
2,4.666667,18.666667,R-2,32.748683,-117.126968,16.666667,16.666667,23.333333,jalapeno taco shop,79.333333,Food & Drinks,2.0,7.0
3,2.000000,16.666667,R-3,32.697974,-117.096250,13.333333,3.333333,12.000000,tacos el campechano inc,46.666667,Only Food,2.0,7.0
4,2.000000,16.666667,R-4,32.718370,-117.157817,13.333333,3.333333,12.000000,up2you cafe llc,46.666667,Only Food,2.0,7.0
5,4.666667,18.666667,R-5,32.677079,-117.107167,16.666667,16.666667,23.333333,awash ethiopian restaurant,79.333333,Food & Drinks,2.0,7.0
6,2.000000,16.666667,R-6,32.890137,-117.150878,13.333333,3.333333,12.000000,gueros taco shop,46.666667,Only Food,3.0,7.0
7,4.666667,18.666667,R-7,32.773069,-117.156144,16.666667,16.666667,23.333333,the fire spot,79.333333,Food & Drinks,3.0,7.0
8,18.000000,4.000000,R-8,33.016118,-117.075608,2.000000,20.000000,18.000000,up2you cafe llc,62.000000,Only Drinks,4.0,8.0
9,4.666667,18.666667,R-9,32.708791,-117.160357,16.666667,16.666667,23.333333,cold beers & cheeseburgers,79.333333,Food & Drinks,2.0,7.0


In [11]:
print(location_counts['row'].min())
print(location_counts['row'].max())
print(location_counts['row'].mean())
print(location_counts['row'].std())
print(location_counts['col'].min())
print(location_counts['col'].max())
print(location_counts['col'].mean())
print(location_counts['col'].std())

-0.0
10.0
2.294110228275949
0.5327487773760726
-0.0
10.0
7.046790263589355
0.31798606536445184


In [12]:
grid = []
for i in range(NUMBER_BLOCKS_ROOT):
    for j in range(NUMBER_BLOCKS_ROOT):
        grid.append([i, j, (lat_min+ i*lat_gap), (long_min + j*long_gap)])    
grid = pd.DataFrame(grid)
grid.columns = ['row','col','lat','lon']
grid
#grid.loc[(grid['row']==142) & (grid['col']==378)]

,row,col,lat,lon
0,0,0,32.100000,-118.426461
1,0,1,32.100000,-118.259176
2,0,2,32.100000,-118.091892
3,0,3,32.100000,-117.924607
4,0,4,32.100000,-117.757323
5,0,5,32.100000,-117.590038
6,0,6,32.100000,-117.422753
7,0,7,32.100000,-117.255469
8,0,8,32.100000,-117.088184
9,0,9,32.100000,-116.920899


In [13]:
location_counts

,am_early,am_peak,id,latitude,longitude,midday,pm_late,pm_peak,title,total_count,type,row,col
0,2.000000,16.666667,R-0,32.767243,-117.096294,13.333333,3.333333,12.000000,c r e a m,46.666667,Only Food,3.0,7.0
1,2.000000,16.666667,R-1,32.697974,-117.096250,13.333333,3.333333,12.000000,snow cones y raspados,46.666667,Only Food,2.0,7.0
2,4.666667,18.666667,R-2,32.748683,-117.126968,16.666667,16.666667,23.333333,jalapeno taco shop,79.333333,Food & Drinks,2.0,7.0
3,2.000000,16.666667,R-3,32.697974,-117.096250,13.333333,3.333333,12.000000,tacos el campechano inc,46.666667,Only Food,2.0,7.0
4,2.000000,16.666667,R-4,32.718370,-117.157817,13.333333,3.333333,12.000000,up2you cafe llc,46.666667,Only Food,2.0,7.0
5,4.666667,18.666667,R-5,32.677079,-117.107167,16.666667,16.666667,23.333333,awash ethiopian restaurant,79.333333,Food & Drinks,2.0,7.0
6,2.000000,16.666667,R-6,32.890137,-117.150878,13.333333,3.333333,12.000000,gueros taco shop,46.666667,Only Food,3.0,7.0
7,4.666667,18.666667,R-7,32.773069,-117.156144,16.666667,16.666667,23.333333,the fire spot,79.333333,Food & Drinks,3.0,7.0
8,18.000000,4.000000,R-8,33.016118,-117.075608,2.000000,20.000000,18.000000,up2you cafe llc,62.000000,Only Drinks,4.0,8.0
9,4.666667,18.666667,R-9,32.708791,-117.160357,16.666667,16.666667,23.333333,cold beers & cheeseburgers,79.333333,Food & Drinks,2.0,7.0


In [14]:
location_counts.to_csv('Location_grid.csv')

In [15]:
total_loc_counts = location_counts[['total_count','row','col']]
total_loc_counts

,total_count,row,col,latitude,longitude
0,46.666667,3.0,7.0,32.767243,-117.096294
1,46.666667,2.0,7.0,32.697974,-117.096250
2,79.333333,2.0,7.0,32.748683,-117.126968
3,46.666667,2.0,7.0,32.697974,-117.096250
4,46.666667,2.0,7.0,32.718370,-117.157817
5,79.333333,2.0,7.0,32.677079,-117.107167
6,46.666667,3.0,7.0,32.890137,-117.150878
7,79.333333,3.0,7.0,32.773069,-117.156144
8,62.000000,4.0,8.0,33.016118,-117.075608
9,79.333333,2.0,7.0,32.708791,-117.160357


In [16]:
weighted_grid = total_loc_counts.groupby(['row','col']).sum().sort_values(by='total_count',ascending=False)
weighted_grid

total_count       latitude      longitude
row   col                                               
 2.0   7.0   120556.452089  181486.681046 -649774.129066
 3.0   7.0    92196.636479   55438.405468 -197889.971543
 2.0   8.0    13548.000000    7538.459537  -27037.295044
 3.0   8.0    12609.301714    5606.675869  -20015.034865
       6.0     7845.333333    3940.100213  -14072.318543
 4.0   8.0     6519.576948    3465.215138  -12292.878848
       7.0     2303.333333    1254.710587   -4451.757785
       6.0      281.333333     165.284483    -586.410564
       9.0      188.000000      99.101903    -350.629862
 5.0   7.0      140.000000      99.973019    -351.715691
 8.0   3.0       93.333333      67.706564    -235.841480
 3.0   10.0      79.333333      32.834946    -116.753615
 8.0   6.0       79.333333      33.999810    -117.346091
 7.0   1.0       62.000000      33.755561    -118.134787
       2.0       62.000000      33.803196    -117.991337
       7.0       46.666667      33.685967    -117.180028
 8.0   4.0       46.666667      34.044693    -117.632456
 10.0 -0.0       46.666667      34.280363    -118.426461
 6.0   7.0       46.666667      33.493747    -117.105767
 3.0   9.0       46.666667      32.854422    -116.848762
 2.0   9.0       46.666667      32.746448    -116.919372
-0.0   8.0       34.092254     192.600000    -702.000000

In [ ]:
# TODO:: Figure out how to merge these two dataframes where the values in grid 
#weighted_grid = pd.merge(weighted_grid, grid, on=['row')
big_grid = pd.merge(grid, 
                weighted_grid.reset_index(), 
                left_on=['row','col'], 
                right_on=['row','col'], 
                how='left')
#big_grid.loc[(big_grid['row']==142) & (big_grid['col']==378)]
big_grid.fillna(0.0)

In [ ]:
big_grid['total_count'].max()

In [ ]:
big_grid.to_csv('Heat_Map.csv')

## Conclusions/Discussion